In [2]:
forestfires_data=pd.read_csv("forestfires.csv")
forestfires_data.head()

<IPython.core.display.Javascript object>

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


In [3]:
forestfires_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   month          517 non-null    object 
 1   day            517 non-null    object 
 2   FFMC           517 non-null    float64
 3   DMC            517 non-null    float64
 4   DC             517 non-null    float64
 5   ISI            517 non-null    float64
 6   temp           517 non-null    float64
 7   RH             517 non-null    int64  
 8   wind           517 non-null    float64
 9   rain           517 non-null    float64
 10  area           517 non-null    float64
 11  dayfri         517 non-null    int64  
 12  daymon         517 non-null    int64  
 13  daysat         517 non-null    int64  
 14  daysun         517 non-null    int64  
 15  daythu         517 non-null    int64  
 16  daytue         517 non-null    int64  
 17  daywed         517 non-null    int64  
 18  monthapr  

In [6]:
forestfires_data=forestfires_data.drop(columns={"month","day"})
forestfires_data.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,small


In [7]:
forestfires_data.size_category.value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [9]:
import sidetable
forestfires_data.stb.freq(['size_category'])

,size_category,Count,Percent,Cumulative Count,Cumulative Percent
0,small,378,0.731141,378,0.731141
1,large,139,0.268859,517,1.000000


In [11]:
forestfires_data.size_category=pd.get_dummies(forestfires_data.size_category)
#small 1 large 0

<IPython.core.display.Javascript object>

In [12]:
x=forestfires_data.drop(columns="size_category")
y=forestfires_data[["size_category"]]

In [30]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.25,random_state=7) # 70% training and 30% test

In [34]:
ytrain=ytrain.size_category.ravel()
ytest=ytest.size_category.ravel()

In [18]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [24]:
grid_search=GridSearchCV(SVC(),
             param_grid={'C': [1,2,5,6,10],
                         'kernel': ('linear', 'rbf'),
                          'gamma':('scale', 'auto'),
                        },cv=5)

In [27]:
grid_search.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 2, 5, 6, 10], 'gamma': ('scale', 'auto'),
                         'kernel': ('linear', 'rbf')})>

In [35]:
grid_result=grid_search.fit(xtrain, ytrain)

In [36]:
grid_search.best_params_

{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}

In [37]:
model1=SVC(C=1, gamma='scale', kernel= 'linear')

In [38]:
model1.fit(xtrain,ytrain)

SVC(C=1, kernel='linear')

In [41]:
train_predict=model1.predict(xtrain)

In [42]:
actual_train=ytrain

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
accuracy_score(actual_train,train_predict)

1.0

In [45]:
test_predict=model1.predict(xtest)

In [46]:
actual_test=ytest

In [47]:
accuracy_score(test_predict,actual_test)

0.9923076923076923

In [48]:
from sklearn.model_selection import cross_val_score

In [63]:
for i in range(3,20):
    score=cross_val_score(model1,x,y.size_category.ravel(),cv=i)
    print(i,score.mean())
    

3 0.9825581395348837
4 0.9844961240310077
5 0.9902912621359224
6 0.9825581395348837
7 0.9864600412545619
8 0.9863882211538462
9 0.9863547758284601
10 0.9883484162895927
11 0.9845261121856868
12 0.9825581395348837
13 0.9902859960552267
14 0.9864328614328615
15 0.9843137254901961
16 0.982421875
17 0.9882352941176471
18 0.9863163656267105
19 0.9863547758284599


In [65]:
score=cross_val_score(model1,x,y.size_category.ravel(),cv=5)
print(score.mean().round(2))
    

0.99
